# Χρησιμοποιήθηκαν 2 free pass

<span style="color:black; font-weight:700;font-size:18px"> Οι βιβλιοθήκες που θα χρειαστούμε. </span>

In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

nltk.download('words')
from nltk.corpus import words

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<span style="color:black; font-weight:700;font-size:18px"> Διαβάζουμε το αρχείο με τους χρήστες και τα review τους. </span>

In [2]:
revDf = pd.read_csv('reviewsRest.csv')
#restF = pd.read_csv('philly_restaurants_categories.csv')

<span style="color:black; font-weight:700;font-size:18px"> Αφαιρούμε τα stopwords, λέξεις που δεν προσφέρουν κάποια πληροφορία. </span>

In [3]:
stop = stopwords.words('english')
revDf['text_without_sw'] = revDf['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
revDf.head

<bound method NDFrame.head of                      review_id                 user_id  \
0       8JFGBuHMoiNDyfcxuWNtrA  smOvOajNG0lS4Pq7d8g4JQ   
1       cvQXRFLCyr0S7EgFb4lZqw  ZGjgfSvjQK886kiTzLwfLQ   
2       8GZbCEMxrbqmRinDbfHWVQ  OkjeFppodgJP_CvB84cJYw   
3       3dVcGYz6GokuEytLrfG8bA  FEI0XkOrUHufSW_rfOTPAA   
4       -xM8cH4AeVzgTJBQHlxfUQ  ZaMB7VbOwaARjxdhXjODxA   
...                        ...                     ...   
155675  v7t0iEY4YnFiVYCMWfi7RA  Ltn-FyXdf8F7GW0nRB6nyw   
155676  fcXXmkr0bZjsZ_l3clVAWw  ZMAZbnWSlKKwxh96A7lMrw   
155677  UmO5D7ZXgT2Bzqh4wH0o3g  rn_B8ffTR_3ZlW9FcE1n3g   
155678  jsBz-uJy6M--L8ogPG52dA  bJ5FtCtZX3ZZacz2_2PJjA   
155679  YAAmRuqBqaFV6kW6ozjBpQ  aFLEltFU8TVgYS1C6tzuUA   

                   business_id  stars  useful  funny  cool  \
0       RZtGWDLCAtuipwaZ-UfjmQ    4.0       0      0     0   
1       EtKSTHV5Qx_Q7Aur9o4kQQ    5.0       3      1     1   
2       FHNIvNgh3fS7VZQq2Y3dsA    3.0       1      0     1   
3       Dv6RfXLYe1atjgz3X

<span style="color:black; font-weight:700;font-size:18px"> Κάνουμε ένα νέο DataFrame με δύο στήλες, το κείμενο του review και την βαθμολογία. Μετά προσθέτουμε μια τρίτη στήλη που είναι οι λέξεις του κειμένου ώς στοιχεία λίστας, η διαδικασία tokenize </span>

In [4]:
revStars = pd.DataFrame()
revStars[['text','stars']] = revDf[['text_without_sw','stars']]
revStars['tokenized_text'] = revStars.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

<span style="color:black; font-weight:700;font-size:18px"> Για το train_set θα χρησιμοποιήσουμε τις πρώτες 110.000 γραμμές.</span>

In [5]:
revStarsTrain = revStars.iloc[:110000]
revStarsTrain.dropna()

,text,stars,tokenized_text
0,Good food--loved gnocchi marinara baked eggpla...,4.0,"[Good, food, --, loved, gnocchi, marinara, bak..."
1,"On scale one things awesome, place bomb. I dra...",5.0,"[On, scale, one, things, awesome, ,, place, bo..."
2,After trying ramen places crazy variety select...,3.0,"[After, trying, ramen, places, crazy, variety,..."
3,"Good food, reasonably priced, nice staff. Not ...",4.0,"[Good, food, ,, reasonably, priced, ,, nice, s..."
4,Picked takeout Village Whiskey (so I can't rev...,5.0,"[Picked, takeout, Village, Whiskey, (, so, I, ..."
...,...,...,...
109995,"The view, feeling classic bar, beer enticing b...",5.0,"[The, view, ,, feeling, classic, bar, ,, beer,..."
109996,The best restaurant I've since I dont know whe...,5.0,"[The, best, restaurant, I, 've, since, I, dont..."
109997,Realized eating beautiful pictures prepared di...,2.0,"[Realized, eating, beautiful, pictures, prepar..."
109998,I back La Fourno many many times past year. I ...,5.0,"[I, back, La, Fourno, many, many, times, past,..."


<span style="color:black; font-weight:700;font-size:18px"> Από εδώ ξεκινάει ο αλγόριθμος. Για αρχή θα δημιουργήσουμε ένα λεξικό freqWords με κλειδιά κάθε λέξη που εμφανίζεται στα review και ως τιμές το πόσες φορές εμφανίζεται η καθεμία λέξη σε όλα τα review. Ύστερα δημιουργούμε από το λεξικό ένα DataFrame wordDf με δύο στήλες, η πρώτη μια λέξη και η δεύτερη το πόσες φορές εμφανίζεται.</span>

In [6]:
global freqWords
freqWords = {}

english_words = set(words.words())

def findFrequency(text):
    for word in text:
        if word not in freqWords:
            freqWords[word] = 1
            continue
        freqWords[word] += 1
#revStars['tokenized_text'].apply(lambda x: [] if x is None else [word for word in x if word in english_words])
revStarsTrain['tokenized_text'].apply(lambda row: findFrequency(row))

wordDf = pd.DataFrame({'word':freqWords.keys(), 'occurances':freqWords.values()})
wordDf.sort_values('occurances', inplace=True, ascending = False)

<span style="color:black; font-weight:700;font-size:18px"> Έδω φιλτράρουμε το DataFrame wordDf και το καθαρίζουμε από σκουπίδια. Για αρχή αφαιρούμε όσες λέξεις εμφανίζονται επάνω από 100000 φορές και κάτω από 100. Δηλαδή βάλαμε ένα άνω και κάτω κατώφλι που αφαιρεί λέξεις που δεν έχουν πληροφορία (πέρα από τα stopwords που ήδη αφαιρέθηκαν), οι πολύ συχνές λέξεις δεν έχουν κάτι ιδιαίτερο, όπως και οι πολύ σπάνιες, ουσιαστικά αφαιρέσαμε τις "ακραίες" τιμές. Μετά, τα review περιείχαν πολλά "σκουπίδια" όπως γράμματα μόνα τους, πεζά και κεφαλαία, τα οποία αφαιρέσαμε. Τέλος βγάλαμε και όλα τα σημεία στήξης.</span>

In [7]:
wordDf = wordDf[wordDf.occurances < 100000]
wordDf = wordDf[wordDf.occurances > 100]
special = ['\'', '.', ',', '/', '!', '@', '#', '%', '^', '&', '*', '(', ')','-','--']
chars = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x', 'y', 'z']
charsC = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
wordDf = wordDf[(wordDf['word'].isin(special)) == False]
wordDf = wordDf[(wordDf['word'].isin(chars)) == False]
wordDf = wordDf[(wordDf['word'].isin(charsC)) == False]
english_words = set(words.words())
wordDf = wordDf[wordDf['word'].isin(words.words())]
freqWords = dict(zip(wordDf.word, wordDf.occurances))
wordDf.head

<bound method NDFrame.head of              word  occurances
1            food       63898
9            good       63016
25          place       58162
130         great       42566
296          like       37362
...           ...         ...
7277     tempting         101
15412    ultimate         101
6537   tremendous         101
5229       museum         101
3959        timed         101

[3124 rows x 2 columns]>

In [8]:
wordDf[wordDf.word=='waitress']

,word,occurances
1498,waitress,6343


<span style="color:black; font-weight:700;font-size:18px"> Εδώ συνεχίζει ο αλγόριθμος. Θέλουμε να διαχωρίσουμε ποια reviews θεωρούνται "καλά" και ποια "κακά". Το κριτήριο το παίρνουμε από τα αστέρια, άμα ένα review έχει πάνω από 3 (>3) θα θεωρηθεί καλό, ενώ άμα έχει από 3 και κάτω θα θεωρηθεί κακό. Τα καλά τα reviews τα κρατάμε στο DataFrame goodReviews και τα κακά στο badReviews </span>

In [9]:
goodReviews = revStarsTrain[revStars.stars>3]
badReviews = revStarsTrain[revStars.stars<=3]

<ipython-input-9-1efb2ae2112a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  goodReviews = revStarsTrain[revStars.stars>3]
<ipython-input-9-1efb2ae2112a>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  badReviews = revStarsTrain[revStars.stars<=3]


<span style="color:black; font-weight:700;font-size:18px"> Και αρχίζουμε να δουλεύουμε με τα καλά review. Χρησιμοποιώντας το λεξικό freqWords προσπαθούμε να δούμε πόσες από τις λέξεις κλειδιά του freqWords εμφανίζονται στα καλά reviews. Για παράδειγμα η λέξη "waitress" εμφανίζεται σε ολόκληρο το train_set 6343 φορές και στα καλά review 2776 και ξέρουμε ότι τις υπόλοιπες εμφανίσεις θα της κάνει στα κακά review. Υπάρχει φυσικά η πιθανότητα κάποιες λέξεις να μην εμφανίζονται καθόλου και όλες οι εμφανίσεις να γίνονται στα κακά reviews (δεν συμβαίνει). Το πλήθος των λέξεων το κρατάμε στο λεξικό goodWords. </span>

In [10]:
global goodWords
goodWords = dict.fromkeys(freqWords.keys(), 0)

def goodWordsDict(text):
    for word in text:
        # check if it's in the dictionary of the ALL text, so we have same words
        if word in freqWords:
            goodWords[word] += 1
            continue
        # if it's not in the other ictionary, continue
        #freqWords[word] += 1
    



goodReviews['tokenized_text'].apply(lambda row: goodWordsDict(row))

0         None
1         None
3         None
4         None
5         None
          ... 
109993    None
109994    None
109995    None
109996    None
109998    None
Name: tokenized_text, Length: 76016, dtype: object

<span style="color:black; font-weight:700;font-size:18px"> Το πλήθος των εμφανίσεων της λέξης waitress στα καλά review. </span>

In [11]:
goodWords['waitress']

2776

<span style="color:black; font-weight:700;font-size:18px"> Επαναλαμβάνουμε την ίδια διαδικασία μα τώρα για κακά reviews. Μετράμε πόσες εμφανίσεις κάνουν οι λέξεις και κρατάμε το πλήθος τους στο λεξικό badWords</span>

In [12]:
global badWords
badWords = dict.fromkeys(freqWords.keys(), 0)

global badDf
badDf = pd.DataFrame()
#english_words = set(words.words())

def badWordsDict(text):
    for word in text:
        # check if it's in the dictionary of the ALL text, so we have same words
        if word in freqWords:
            badWords[word] += 1
            continue
    
    

badReviews['tokenized_text'].apply(lambda row: badWordsDict(row))

2         None
8         None
11        None
13        None
20        None
          ... 
109979    None
109980    None
109986    None
109997    None
109999    None
Name: tokenized_text, Length: 33984, dtype: object

<span style="color:black; font-weight:700;font-size:18px"> Το πλήθος των εμφανίσεων της λέξης waitress στα κακά review. Πράγματι επαληθεύεται ότι οι υπόλοιπες φορές που δεν εμφανίστηκαν στα καλά review, είναι εδώ.</span>

In [13]:
badWords['waitress']

3567

<span style="color:black; font-weight:700;font-size:18px"> Εδώ βάζουμε τις πληροφορίες για τις εμφανίσεις των λέξεων στα καλά και στα κακά review σε ένα DataFrame allWords. Έχουμε 4 στήλες αρχικά, η πρώτη στήλη ονομάζεται G_word και έχει τις λέξεις όλων των review, η δεύτερη G_occurances και περιέχει το πλήθος των εμφανίσεων των λέξεων στα καλά review, η τρίτη B_word περιέχει, όπως και η G_word, τις λέξεις όλων των review και η τέταρτη B_occurances περιέχει το πλήθος των εμφανίσεων των λέξεων στα κακά review. Μετά βρίσκουμε μια πέμπτη στήλη diff_per που περιέχει την διαφορά σε ποσοστό εμφανίσεων των καλών λέξεων προς το ολικό πλήθος από το 50%. Δηλαδή άμα μια λέξη εμφανίζεται ισόποσα και στα G_occurances και στα Β_occurances η στήλη diff_per θα έχει την τιμή 0. Αν μια λέξη εμφανίζεται κατά 70% στα καλά review η στήλη diff_per θα έχει την τιμή 20 (70-50). Αν μια λέξη εμφανίζεται κατά 10% στα καλά review η στήλη diff_per θα έχει την τιμή -40 (10-50). </span>

In [14]:
allWords = pd.DataFrame({'G_word':goodWords.keys(), 'G_occurances':goodWords.values(),
                      'B_word':badWords.keys(), 'B_occurances':badWords.values()})

allWords['diff_per'] = allWords.G_occurances/(allWords.G_occurances+allWords.B_occurances)*100 - 50
allWords.head()

,G_word,G_occurances,B_word,B_occurances,diff_per
0,food,39424,food,24474,11.698332
1,good,41572,good,21444,15.970547
2,place,39514,place,18648,17.937829
3,great,34373,great,8193,30.752244
4,like,21788,like,15574,8.315936


<span style="color:black; font-weight:700;font-size:18px"> Επόμενο βήμα, δημιουργούμε δύο DataFrame με καλές και κακές λέξεις. Το κριτήριο είναι η αναλογία εμφανίσεων της λέξης σε σχέση με το ολικό πλήθος εμφανίσεων, δηλαδή σύμφωνα με την στήλη diff_per. Στο DataFrame goodDf κρατάμε όσες λέξεις περιέχονται κατά 80% στα καλά review (diff_per >= 30) και στο DataFrame badDf κρατάμε όσες λέξεις περιέχονται κατά 30% στα κακά review (diff_per<=-20). Όσες λέξεις είναι μοιρασμένες μεταξύ καλών και κακών review κατά 30-80% θεωρείται ότι δεν προσφέρουν κάποια ιδιαίτερη πληροφορία, καθώς είναι ουσιαστικά κοινές και αγνοούνται.</span>

In [15]:
goodSer = allWords[allWords.diff_per >= 30]['G_word']
goodDf = goodSer.to_frame(name='G_word')

badSer = allWords[allWords.diff_per <= -20]['B_word']
badDf = badSer.to_frame(name='B_word')

<span style="color:black; font-weight:700;font-size:18px"> Εδώ επεξεργαζόμαστε το test_set. Παίρνουμε τις υπόλοιπες από τις 110.000 γραμμές και προσθέτουμε μια τρίτη στήλη label. Άμα η τιμή stars είναι μεγαλύτερη του 3 θα βάλουμε label 1 (καλό review), αλλιώς την τιμή 0 (κακό review).</span>

In [16]:
test_data = pd.DataFrame()
test_data = revStars[['tokenized_text','stars']].iloc[110000:]
# ######## 1 is for GOOD 0 is for BAD
test_data['label'] = test_data['stars'].apply(lambda star: 1 if(star>3) else 0)
test_data.head()

,tokenized_text,stars,label
110000,"[Awesome, staff, ,, awesome, beer, ,, awesome,...",5.0,1
110001,"[I, found, tiny, worm, chicken, wing, platter,...",1.0,0
110002,"[What, let, down, ., Food, drinks, okay, ., Th...",3.0,0
110003,"[This, great, place, enjoy, dinner, work, ., I...",4.0,1
110004,"[I, think, I, rediscovered, love, pizza, !, I,...",5.0,1


<span style="color:black; font-weight:700;font-size:18px"> Εδώ υλοποιείται ένας αλγόριθμος για την αξιολόγηση των λιστών. Θα παίρνουμε τα test review μας, και η στήλη tokenized_text περιέχει το review σε μορφή λίστας, και μετά ανάλογα με το που βρίσκεται κάθε λέξη του review, στην λίστα με τις καλές λέξεις goodList ή στην λίστα με τις κακές λέξεις badList, θα γίνεται μια ψηφοφορία. Αν στο τέλος οι περισσότερες λέξεις βρισκόντουσταν στην goodList θα προβλεφτεί ώς label 1 (good) αν βρίσκονται στο badList θα προβλεφτεί ώς label 0 (bad).</span>

In [17]:
ll = []
def predict(text):
    goodVote = 0
    badVote = 0
    goodList = goodDf['G_word'].values
    badList = badDf['B_word'].values
    for word in text:
        # check if it's in the dictionary of the ALL text, so we have same words
        if (word in goodList):
            goodVote += 1
        if (word in badList):
            badVote += 1
    if(goodVote > badVote):
        return 1
    else:
        return 0

In [18]:
test_data['predicted'] = test_data['tokenized_text'].apply(lambda row: predict(row))

In [19]:
# train has 110k samples
acc = metrics.accuracy_score(test_data['predicted'], test_data['label'])
print("Accuracy",acc)
prec = metrics.precision_score(test_data['predicted'], test_data['label'])
print("Precision",prec)
rec = metrics.recall_score(test_data['predicted'], test_data['label'])
print("Recall",rec)

Accuracy 0.7893826619964974
Precision 0.9160237892948173
Recall 0.7903471733520995
